### Some tests for an Ising model implementation and visualization

The Ising model Hamiltonian is $\mathcal{H} = -J\sum_{\langle i,j \rangle} \sigma_i \sigma_j - \sum_j h\sigma_j$. The partition function is $Z=\sum_{\{\sigma_i\}} e^{-\beta U}$ with $U=-\epsilon\sum_{\langle i,j \rangle} \sigma_i \sigma_j$ (8.38 and 8.39). The partition function sums over each possible lattice states but there are $2^N$ possible lattice states so for a $10\times 10$ lattice, we have $2^{10^2}\approx 10^{30}$ configurations. This means writing a program that attempts to analyze each possible lattice state will not succeed.

The Metropolis algorithim is a method of importance sampling that generates sample satets for a Monte Carlo simulation (integration) with the correct Boltzmann probabilities. This is a natural choice because we should *probably* sample the most likely states the most --- we don't need to sample extremely rare states. (Think of choosing a more clever volume to integrate over instead of a rectangle to approximate the volume of some other shape). **Schroeder uses periodic (torus) BC**.

Why is Metropolis done 100 times per site? Here I implement the pseudo code in Schroeder.

In [ ]:
import pygame
from pygame.locals import *
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Im just trying to get pygame to work quickly. Then Ising implementation should be good

In [53]:
import pygame
from pygame.locals import *
import sys
import numpy as np

# Initializing Pygame
pygame.init()
 
# Initializing surface
surface = pygame.display.set_mode((500,500))
 
# Initializing Color
color = (255,0,0)
 
# Drawing Rectangle
#pygame.draw.rect(surface, color, pygame.Rect(10, 20, 490, 490)) #rect: x,y,width,height . . . draws from x,y at top left
# Draw lattice
latticeSize = 500 
c = 0
for x in np.arange(0,latticeSize,10):
    c += 1
    c = c%2
    color = (255,0,0)
    if(c!=1):
        color = (0,0,255)
    for y in np.arange(0,latticeSize,10):
        pygame.draw.rect(surface, color, pygame.Rect(x, y, 10, 10)) #each site will be a square of size 5 
pygame.display.flip()
while True:
    for event in pygame.event.get():
        # check if the event is the X button
        if event.type==pygame.QUIT:
            # if it is quit the game
            pygame.display.quit()
            pygame.quit()
            sys.exit()

SystemExit: 

C:\Users\Noah\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
